# Regression model for IBNR estimates

In [2]:
import os
import itertools
import pandas as pd
import re
import math
import time
import numpy as np

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [3]:
def columnas(valores,variable):
    y = [re.findall("\\d+", j)[0] for j in valores]
    y = [int(i) for i in y]
    todas = list(set(y))
    df = pd.DataFrame()
    df[f"y_{variable}"] = y
    for k in todas:
        #print(k)
        df[f"{variable}_{k}"] = ([1 if k == j else 0 for j in y])
    return df

def matrix_X(df_triangulo):
    k = len(df_triangulo.columns)
    alpha = [f'a_{i}' for i in range(1,k+1)]
    mu    = [f'u_{i}' for i in range(1,k+1)]
    lists = [alpha, mu]
    df    = pd.DataFrame(list(itertools.product(*lists)), columns=['a', 'u'])

    alpha    = columnas(valores  = df.a, variable = 'a')
    mu       = columnas(valores=df.u, variable = 'u')
    df_col= pd.concat([alpha, mu], axis=1)


    df_col['y_a'] = df_col['y_a'].astype(str) + df_col['y_u'].astype(str) 
    df_col['y_a'] = [int(i) for i in df_col['y_a']]
    df_col = df_col.drop(['y_u', 'u_1'], axis=1)
    df_col['a_1'] = 1
    df_col.rename(columns={'a_1': 'b0'}, inplace=True)
    df_col.rename(columns={'y_a': 'y_ii'}, inplace=True)
    #df_col = df_col.drop(['y_ii'], axis=1)
    return df_col

def matrix_y(df_triangulo):
    k = len(df_triangulo.columns)
    d0 = pd.DataFrame()
    for i in range(k):
        for j in range(k):
            d1 = pd.DataFrame({'y_ii': [int(f'{i+1}{j+1}')], 'Y': [math.log(df_triangulo.iloc[i, j])]})
            d0 = pd.concat([d0, d1], axis=0)
    return d0

def triangulo(df, grcode, entreno):
    
    if entreno:
        df_trinagulo = df[(df['GRCODE']== grcode ) & (df['DevelopmentYear']<=1997)].copy()
    else: 
        df_trinagulo = df[df['GRCODE']== grcode].copy()
        
    df_g         = df_trinagulo.groupby(["AccidentYear", "DevelopmentLag"]).agg({'IncurLoss_B': ['max']})
    df_g.columns = ['Pagos']
    df_g         = df_g.reset_index()
    pivot_data   = df_g.pivot(index='AccidentYear',columns='DevelopmentLag',values='Pagos').reset_index()
    pivot_data   = pivot_data.drop('AccidentYear', axis=1).cumsum(axis=1)
    
    return pivot_data


In [4]:
input = pd.read_csv(os.path.normpath(os.getcwd() + os.sep + os.pardir)+"/data/ppauto_pos.csv")

input = input[input.DevelopmentYear <= 1997]

cleaning_cond = np.array(['Adriatic Ins Co', 'Aegis Grp', 'Agency Ins Co Of MD Inc',
       'Allegheny Cas Co', 'American Modern Ins Grp Inc',
       'Armed Forces Ins Exchange', 'Auto Club South Ins Co',
       'Baltica-Skandinavia Rein Co Of Amer', 'Bancinsure Inc',
       'Bell United Ins Co', 'Century-Natl Ins Co', 'Co-Operative Ins Co',
       'Consumers Ins Usa Inc', 'Cornerstone Natl Ins Co',
       'Federated Natl Ins Co', 'First Amer Ins Co',
       'Florists Mut Ins Grp', 'Harbor Ins Co', 'Homestead Ins Co',
       'Inland Mut Ins Co', 'Interstate Auto Ins Co Inc', 'Lancer Ins Co',
       'Lumber Ins Cos', 'Manhattan Re Ins Co', 'Mennonite Mut Ins Co',
       'Middle States Ins Co Inc', 'National Automotive Ins',
       'Nevada General Ins Co', 'New Jersey Citizens United Rcp Exch',
       'Nichido Fire & Marine Ins Co Ltd', 'Northwest Gf Mut Ins Co',
       'Ocean Harbor Cas Ins Co', 'Overseas Partners Us Reins Co',
       'Pacific Ind Ins Co', 'Pacific Pioneer Ins Co',
       'Pacific Specialty Ins Co', 'Penn Miller Grp',
       'Pennsylvania Mfg Asn Ins Co', 'Pioneer State Mut Ins Co',
       'Protective Ins Grp', 'San Antonio Reins Co',
       'Seminole Cas Ins Co', 'Southern Group Ind Inc',
       'Southern Mut Ins Co', 'Southland Lloyds Ins Co', 'Star Ins Grp',
       'Sterling Ins Co', 'Usauto Ins Co', 'Vanliner Ins Co',
       'Wea Prop & Cas Ins Co', 'Wellington Ins Co', 'State Farm Mut Grp', 'United Services Automobile Asn Grp',
       'US Lloyds Ins Co', 'Toa-Re Ins Co Of Amer', 'FL Farm Bureau Grp'])

input = input[~input.GRNAME.isin(cleaning_cond)]

Lista_entidades_ceros = input[input.IncurLoss_B <= 0]["GRCODE"].unique()
input = input[~input.GRCODE.isin(Lista_entidades_ceros)]
#input.IncurLoss_B = input.IncurLoss_B+1 #deal with NaN from log transformation.

In [5]:
df_data        = input #pd.read_csv('medmal_pos.csv')
df_trg_entreno = triangulo(df_data, grcode=43, entreno=True)
df_trg_prueba  = triangulo(df_data, grcode=43, entreno=False)
df_trg_entreno

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
0,607.0,1254.0,1836.0,2434.0,3048.0,3663.0,4278.0,4892.0,5506.0,6120.0
1,2254.0,5113.0,8092.0,10856.0,13682.0,16699.0,19689.0,22667.0,25645.0,NaN
2,5843.0,13267.0,21574.0,30245.0,39311.0,48237.0,57004.0,65769.0,NaN,NaN
3,11422.0,27515.0,46163.0,65258.0,83911.0,102380.0,120787.0,NaN,NaN,NaN
4,19933.0,44095.0,72834.0,101163.0,129234.0,156956.0,NaN,NaN,NaN,NaN
5,24604.0,56734.0,90309.0,123078.0,156800.0,NaN,NaN,NaN,NaN,NaN
6,40735.0,84679.0,127190.0,168902.0,NaN,NaN,NaN,NaN,NaN,NaN
7,43064.0,86769.0,129678.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,41837.0,83141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,44436.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
Y = matrix_y(df_trg_entreno)
X = matrix_X(df_trg_entreno)

Y_X          = pd.merge(Y, X, on='y_ii', how='inner')
data_entreno = Y_X[Y_X['Y'].notna()]
data_entreno = data_entreno.drop(['y_ii'], axis=1)
data_entreno.head()

,Y,b0,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,u_10
0,6.408529,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7.134094,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,7.515345,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,7.797291,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,8.022241,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [7]:
Y_prueba      = matrix_y(df_trg_prueba)                   
x_prueba      = Y_X[Y_X['Y'].notna()].drop(['Y'], axis=1)
data_prueba_  = pd.merge(Y_prueba, x_prueba, on='y_ii', how='inner')
data_prueba=data_prueba_.drop(['y_ii'], axis=1)
y_ii = data_prueba_['y_ii']

x_entreno = data_entreno.drop('Y', axis=1)  # Features
y_entreno = data_entreno['Y']  # Target variable
x_prueba  = data_prueba.drop('Y', axis=1)  # Features
y_prueba  = data_prueba['Y']  # Target variable

In [8]:
Regresion_lineal = LinearRegression()
Regresion_lineal.fit(x_entreno, y_entreno)
LR_coef = Regresion_lineal.coef_
y_pred = Regresion_lineal.predict(x_prueba)

mse = mean_squared_error(y_prueba, y_pred)   # Considerar que se debe aplicar la exponencial a los rsultados
mape = mean_absolute_percentage_error(y_prueba, y_pred)   # COnsiderar que se debe aplicar la exponencial a los rsultados
[mse, mape]

[0.002230701729399499, 0.0038343781268415497]

In [9]:
alpha = 0.00001
ridge_model = Ridge(alpha = alpha) #aplicación regresión de ridge
ridge_model.fit(x_entreno, y_entreno) #entrenamiento regresión de ridge
ridge_coef = ridge_model.coef_ #coeficientes regresión de ridge
y_pred = ridge_model.predict(x_prueba)

mse = mean_squared_error(y_prueba, y_pred)   # COnsiderar que se debe aplicar la exponencial a los rsultados
mape = mean_absolute_percentage_error(y_prueba, y_pred)   # COnsiderar que se debe aplicar la exponencial a los rsultados
[mse, mape]

[0.002230702303162153, 0.003834869524506071]

In [10]:
lasso_model = Lasso(alpha = alpha) #aplicación regresión de lasso
lasso_model.fit(x_entreno, y_entreno) #entrenamiento regresión de lasso
lasso_coef = lasso_model.coef_ #coeficientes regresión de lasso
y_pred = lasso_model.predict(x_prueba)

mse = mean_squared_error(y_prueba, y_pred)   # Considerar que se debe aplicar la exponencial a los rsultados
mape = mean_absolute_percentage_error(y_prueba, y_pred)   # Considerar que se debe aplicar la exponencial a los rsultados
[mse, mape]

[0.0022309333284786215, 0.0038438188771605904]

In [173]:
np.exp(y_pred)

array([   523.69588732,   1138.91125798,   1800.60267558,   2481.25822179,
         3197.47927039,   3891.61694156,   4565.05488313,   5164.35159533,
         5680.89885172,   6130.        ,   2295.86135051,   4992.94037277,
         7893.76848391,  10877.73455923,  14017.61834235,  17060.68950208,
        20013.01389388,  22640.30616852,  24904.82821343,   6338.44464237,
        13784.57638498,  21793.22133009,  30031.39467575,  38700.02770899,
        47101.37916013,  55252.19572384,  62505.6592827 ,  13208.63690653,
        28725.57459323,  45414.72929312,  62582.19649294,  80646.69538382,
        98154.20820492, 115139.6332668 ,  20832.59948126,  45305.8400049 ,
        71627.89564198,  98704.34348534, 127195.58545728, 154808.35164171,
        25717.11423791,  55928.47230123,  88422.12785315, 121847.05415536,
       157018.49424527,  38030.54940143,  82707.20070384, 130758.92070899,
       180187.806828  ,  40167.54770632,  87354.65256788, 138106.47672126,
        39993.8249981 ,  

In [55]:
df_CV = input[input["GRCODE"].isin(list(input["GRCODE"].unique()[0:10]))]
lista_aseguradoras = df_CV["GRCODE"].unique()
mejore_modelos_test_full = {}
mejores_mape = {}

for i in range(len(lista_aseguradoras)): #recorre las aseguradoras de test
    print("aseguradora de test:", i)
    conj_test = lista_aseguradoras[i] #codigo de aseguradora de testeo
    datos_test = df_CV[df_CV["GRCODE"].isin([conj_test])] #datos de aseguradora de testeo
    conj_entre_valid = np.delete(lista_aseguradoras, i, axis=0) #Conjunto de validación y entrenamiento

    for j in range(len(conj_entre_valid)): #recorre los datos de entrenamiento y validación

        conj_vali = conj_entre_valid[j] #datos de aseguradora de validación
        conj_entre = np.delete(conj_entre_valid, j, axis=0) #aseguradoras de entrenamiento
        datos_train = df_CV[df_CV["GRCODE"].isin(conj_entre)] #datos de aseguradoras de entrenamiento
        datos_validacion = df_CV[df_CV["GRCODE"].isin([conj_vali])] #datos de aseguradora de validación

        #Se crea la clase que calculará las regresiones
        df_data        = input #pd.read_csv('medmal_pos.csv')
        df_trg_entreno = triangulo(datos_train, entreno=True, grcode=43)
        df_trg_prueba  = triangulo(datos_validacion, entreno=False, grcode=conj_vali)

        Y_prueba      = matrix_y(df_trg_prueba)                   
        x_prueba      = Y_X[Y_X['Y'].notna()].drop(['Y'], axis=1)
        data_prueba_  = pd.merge(Y_prueba, x_prueba, on='y_ii', how='inner')
        data_prueba=data_prueba_.drop(['y_ii'], axis=1)
        y_ii = data_prueba_['y_ii']

        x_entreno = data_entreno.drop('Y', axis=1)  # Features
        y_entreno = data_entreno['Y']  # Target variable
        x_prueba  = data_prueba.drop('Y', axis=1)  # Features
        y_prueba  = data_prueba['Y']  # Target variable

        Regresion_lineal.fit(x_entreno, y_entreno)
        LR_coef = Regresion_lineal.coef_
        y_pred_1 = Regresion_lineal.predict(x_prueba)
        mape_1 = mean_squared_error(y_prueba, y_pred_1)
        Regresion_lineal_sum = [Regresion_lineal.intercept_, Regresion_lineal.coef_, Regresion_lineal.score(x_prueba, y_prueba)]

        ridge_model.fit(x_entreno, y_entreno)
        ridge_coef = Regresion_lineal.coef_
        y_pred_2 = ridge_model.predict(x_prueba)
        mape_2 = mean_squared_error(y_prueba, y_pred_2)
        ridge_sum = [ridge_model.intercept_, ridge_model.coef_, ridge_model.score(x_prueba, y_prueba)]

        lasso_model.fit(x_entreno, y_entreno)
        lasso_coef = Regresion_lineal.coef_
        y_pred_3 = lasso_model.predict(x_prueba)
        mape_3 = mean_squared_error(y_prueba, y_pred_3)
        lasso_sum = [lasso_model.intercept_, lasso_model.coef_, lasso_model.score(x_prueba, y_prueba)]

        results = [mape_1, mape_2, mape_3]
        models = [Regresion_lineal_sum, ridge_sum, lasso_sum]

        if (mape_1<mape_2) & (mape_1<mape_3):
            modelo_test=Regresion_lineal
            if (mape_2<mape_1) & (mape_2<mape_3):
                modelo_test=ridge_model
                if (mape_3<mape_1) & (mape_3<mape_2):
                    modelo_test=lasso_model
                    
        y_pred = modelo_test.predict(x_prueba)
        mejores_mape[i,j] = mean_squared_error(y_prueba, y_pred)

        mejore_modelos_test_full["modelo_"+str(i)+"-"+str(j)] = modelo_test #se guardan los mejores modelos


aseguradora de test: 0
aseguradora de test: 1
aseguradora de test: 2
aseguradora de test: 3
aseguradora de test: 4
aseguradora de test: 5
aseguradora de test: 6
aseguradora de test: 7
aseguradora de test: 8
aseguradora de test: 9


In [71]:
list(dict(sorted(mejores_mape.items(), key=lambda item: item[1])).keys())[0]

(1, 0)

In [57]:
mejore_modelos_test_full #winner is number 10

{'modelo_0-0': LinearRegression(),
 'modelo_0-1': LinearRegression(),
 'modelo_0-2': LinearRegression(),
 'modelo_0-3': LinearRegression(),
 'modelo_0-4': LinearRegression(),
 'modelo_0-5': LinearRegression(),
 'modelo_0-6': LinearRegression(),
 'modelo_0-7': LinearRegression(),
 'modelo_0-8': LinearRegression(),
 'modelo_1-0': LinearRegression(),
 'modelo_1-1': LinearRegression(),
 'modelo_1-2': LinearRegression(),
 'modelo_1-3': LinearRegression(),
 'modelo_1-4': LinearRegression(),
 'modelo_1-5': LinearRegression(),
 'modelo_1-6': LinearRegression(),
 'modelo_1-7': LinearRegression(),
 'modelo_1-8': LinearRegression(),
 'modelo_2-0': LinearRegression(),
 'modelo_2-1': LinearRegression(),
 'modelo_2-2': LinearRegression(),
 'modelo_2-3': LinearRegression(),
 'modelo_2-4': LinearRegression(),
 'modelo_2-5': LinearRegression(),
 'modelo_2-6': LinearRegression(),
 'modelo_2-7': LinearRegression(),
 'modelo_2-8': LinearRegression(),
 'modelo_3-0': LinearRegression(),
 'modelo_3-1': Linea

In [48]:
winner = mejore_modelos_test_full['modelo_1-0']

In [50]:
winner.coef_

array([0.        , 1.47921164, 2.49497522, 3.22928449, 3.68495836,
       3.89560542, 4.28685133, 4.34152385, 4.33719212, 4.44254939,
       0.77693372, 1.2349843 , 1.55563044, 1.80923086, 2.00569641,
       2.16530556, 2.28865714, 2.38398713, 2.46006154])

In [58]:
df_data        = input
df_trg_prueba  = triangulo(df_data, grcode=43, entreno=False)

Y_prueba      = matrix_y(df_trg_prueba)                   
x_prueba      = Y_X[Y_X['Y'].notna()].drop(['Y'], axis=1)
data_prueba_  = pd.merge(Y_prueba, x_prueba, on='y_ii', how='inner')
data_prueba=data_prueba_.drop(['y_ii'], axis=1)
y_ii = data_prueba_['y_ii']

X_test  = data_prueba.drop('Y', axis=1)  # Features
Y_test  = data_prueba['Y']  # Target variable


Y_pred = winner.predict(X_test)

In [ ]:
mejores_mape[(1,0)]

0.002230701729399499

In [68]:
pd.DataFrame({"Prueba" : Y_test, "Ajustado" : Y_pred, "Percentage difference" : np.round(100*(Y_test-Y_pred)/Y_pred, 2)})

,Prueba,Ajustado,Percentage difference
0,6.408529,6.259256,2.38
1,7.134094,7.036190,1.39
2,7.515345,7.494240,0.28
3,7.797291,7.814886,-0.23
4,8.022241,8.068487,-0.57
5,8.206038,8.264952,-0.71
6,8.361241,8.424561,-0.75
7,8.495356,8.547913,-0.61
8,8.613594,8.643243,-0.34
9,8.719317,8.719317,0.00


Ejemplo de como utilizar la clase para entrenar y evaluar las tres regresiones

Evaluación del modelo en el conjunto de validación

Acá se evaluan los tres modelos en el conjunto de validación o aseguradora de validación


Evaluación del modelo en el conjunto de testeo
Acá se evalua el mejor modelo que escogieron los datos de validación en el conjunto de test o aseguradora de test